In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append("../")

In [2]:
# choose whether to work on a remote machine
location = "remote"
# location = "local"

if location == "remote":
    # change this line to the where the GitHub repository is located
    os.chdir("/lustre_scratch/orlando-code/coralshift/")

# Data storage setup

In [3]:
# # import necessary packages

from __future__ import annotations

from pathlib import Path
import xarray as xa
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cartopy.crs as ccrs

from rasterio.plot import show
import rioxarray as rio
import rasterio

from coralshift.processing import spatial_data
from coralshift.utils import file_ops, directories, utils
from coralshift.dataloading import data_structure, climate_data, bathymetry, reef_extent
from coralshift.utils import directories
from coralshift.plotting import spatial_plots



ERROR 1: PROJ: proj_create_from_database: Open of /home/jovyan/lustre_scratch/conda-envs/coralshift/share/proj failed


## Specify your area of interest

The availability of high-resolution (30m) bathymetric data means that areas of interest are currently confided to 4 areas on the Great Barrier Reef (GBR). The following code downloads the specified area of bathymetry data:

| Reef Area Name                	| Latitudes 	| Longitudes 	|
|-------------------------------	|-----------	|------------	|
| Great Barrier Reef A 2020 30m 	| 10-17°S   	| 142-147°E  	|
| Great Barrier Reef B 2020 30m 	| 16-23°S   	| 144-149°E  	|
| Great Barrier Reef C 2020 30m 	| 18-24°S   	| 148-154°E  	|
| Great Barrier Reef D 2020 30m 	| 23-29°S   	| 150-156°E  	|


![bathymetry_regions.png](https://github.com/orlando-code/coralshift/blob/dev-setup/bathymetry_regions.png?raw=true)


Due to the computational load required to run ML models on such high resolution data, bathymetric data is currently upsampled to 4km grid cells and areas are limited to a quarter of the GBR's total area.


In [4]:
# choose resolution (should be above 1000m for processing in decent time)
# native resolutions are 1 (1, "m") or 1/12 degrees (1/12, "d"), or 4000m (4000, "m") or 1000m (1000, "m")
target_resolution_m, target_resolution_d = spatial_data.choose_resolution(
    resolution=4000, unit="m")

print(f"Data will be resampled to {target_resolution_d:.05f} degrees (~{target_resolution_m}m).")

Data will be resampled to 0.03691 degrees (~4000m).


## Bathymetry

In [10]:
# select your area. Region "A" is used in the report
area_name = "D"

reef_areas = bathymetry.ReefAreas()
file_name = reef_areas.get_short_filename(area_name)


bath_dir = directories.get_bathymetry_datasets_dir()
_, xa_bath = bathymetry.generate_bathymetry_xa_da(area_name)
_, xa_bath_upsampled = spatial_data.upsample_and_save_xa_a(
    bath_dir, xa_d=xa_bath, name=file_name, target_resolution_d=target_resolution_d)

Great_Barrier_Reef_D_2020_30m_MSL_cog already exists in /lustre_scratch/orlando-code/datasets/bathymetry
Writing bathymetry_D_0-00030d to file at /lustre_scratch/orlando-code/datasets/bathymetry/bathymetry_D_0-00030d.nc


In [ ]:
# # compute dasked arrays for plotting
# xa_bath_upsampled = xa_bath_upsampled.compute()

# # N.B. native resolution not plotted since so high (takes ~10 minutes)
# spatial_plots.plot_spatial(xa_bath_upsampled, val_lims=(-50,0), name="depth", 
#     title=f"Bathymetry at {target_resolution_m}m resolution")

## Calculate slopes

In [ ]:
# calculate absolute gradients from bathymetry and save to file
grads, grads_path = bathymetry.generate_gradient_magnitude_nc(extract_variable(xa_bath_upsampled), sigma=1)

In [ ]:
### Display different resolutions
fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(16,9), subplot_kw=dict(projection=ccrs.PlateCarree()))

ax1 = spatial_plots.plot_spatial(xa_bath_upsampled, 
    fax= (fig,ax_left), val_lims=(-50,0), name="depth", title=f"Bathymetry at {target_resolution_m}m resolution")
ax2 = spatial_plots.plot_spatial(grads, 
    fax=(fig, ax_right), val_lims=(0,10), name="gradient magnitude", 
    title=f"Absolute seafloor gradients at {target_resolution_m}m resolution")

## Coral ground truth: Allen Coral Atlas


There is currently no API for accessing data directly from your local machine. Please follow the instructions* below:
1. Make an account on the [Allen Coral Atlas](https://allencoralatlas.org/atlas/#6.00/-13.5257/144.5000) webpage
2. Generate a geojson file using the code cell below (generated in the `reef_baseline` directory)

*Instructions correct as of 30.06.23

In [ ]:
# generate geojson file in reef_baseline directory for download from the Allen Coral Atlas
geojson_path = reef_extent.generate_area_geojson(
    area_class = reef_areas, area_name=file_name, save_dir=directories.get_reef_baseline_dir())

print(f"geoJSON file saved at {geojson_path} for upload to GEE")

3. Upload the geojson file via:

    \> My Areas > Upload a GeoJSON or KML file
4. Specify a region name and navigate to the "Download data" tab when it becomes available.
4. Select "Benthic Map (OGC GeoPackage (.gpkg))". Sign the terms and conditions 
and select "Prepare Download". After ~two minutes a sequence of emails will arrive notifying you that your download is ready.
5. Download the file and unzip it using a unzipping utility. Then, add the `benthic.gpkg` file to the `reef_baseline` directory.
6. Continue with the subsequent code cells.

----

You have now downloaded:

**`benthic.gpkg`**

This is a dataframe of Shapely objects ("geometry" polygons) defining the boundaries of different benthic classes:

| Class           	| Number of polygons 	|
|-----------------	|--------------------	|
| Coral/Algae     	| 877787             	|
| Rock            	| 766391             	|
| Rubble          	| 568041             	|
| Sand            	| 518805             	|
| Microalgal Mats 	| 27569              	|

In [ ]:
# read .gpkg file and save to .pkl format for faster reading later
benthic_df = file_ops.check_pkl_else_read_gpkg(directories.get_reef_baseline_dir(), filename = "benthic.pkl")
benthic_df.head()

### Rasterize polygons

Rasterized arrays are necessary to process the geospatial data e.g. to align different gridcells. Doing this locally through rasterio requires such significant compute that cloud computing is the only reasonable option. A JavaScript file (`rasterization.js`) for use in Google Earth Engine (GEE) is accessible [here](https://code.earthengine.google.com/ae68c68309b04643e8f5f5dc45f0dbca). Visit [this page](https://developers.google.com/earth-engine/guides/getstarted) for information regarding setting up a GEE account and getting started.

GEE requires shapefile (.shp) format to ingest data. This is generated in the following cell:


In [ ]:
# process df to gpd.GeoDataFrame. 
# We are interested only in the "Coral/Algae" class, so gdf is limited to these rows by default
gdf_coral = reef_extent.process_benthic_pd(benthic_df)
# save as shapely file (if not already present) for rasterisation in GEE
reef_extent.generate_coral_shp(gdf_coral)

1. Ingest the shapefile (and all accompanying files: .cpg, .dbf, .prj, .shx) as a GEE asset.
2. Import the subsequent `Table` into the script.
3. Update the `resolution` variable as desired (usually that matching the target resolution specified above).
3. Run the script, and submit the `coral_raster_Xm` task. Sit back and wait! After ~1 hour (depending on the chosen resolution) the rasters will be available to download from your Google Drive as GeoTIFFS: after this, add them to the `reef_baseline` directory and carry on with the following cells.

In [ ]:
# process all tifs in folder, casting to nc files for future use
reef_extent.process_reef_extent_tifs()


In [ ]:
# TODO: automated visualisation

gt_tif_dict['coral_raster_1000m'].plot()

## Global Ocean Physics Reanalysis

The dataset metadata can be accessed [here](https://doi.org/10.48670/moi-00021).

### Download data

You're required to set up an account with the [Copernicus Marine Service](https://marine.copernicus.eu/). 


**Warning:**  this is a large amount of data for which the only way to gather it is to query the copernicus API via motu. Requests are queued, and request sizes are floated to the top of the queue. The following functions take advantage of this by splitting a single request up by date adn variable before amalgamating the files, but this can still take a **very long time**, and vary significantly depending on overall website traffic. For those who aren't interested in the entire database, it's highly recommended that you use the toy dataset provided as a `.npy` file in the GitHub repository.


In [ ]:
# download monthly data. Can be adjusted to specify subset of variables, dates, and depths to download.
# Values generated here are those reported in the accompanying paper.
xa_cmems_monthly, cmems_monthly_path = climate_data.download_reanalysis(download_dir=directories.get_monthly_cmems_dir(),
    region = reef_areas.get_short_filename(area_name),
    final_filename = f"cmems_gopr_monthly_{area_name}",
    lat_lims = reef_areas.get_lat_lon_limits(area_name)[0], lon_lims = reef_areas.get_lat_lon_limits(area_name)[1], 
    product_id = "cmems_mod_glo_phy_my_0.083_P1M-m")   


 variable loop:   0%|          | 0/7 [00:00<?, ?it/s]

2023-06-23 17:14:23.357 [ INFO] Asynchronous mode set
2023-06-23 17:14:23.357 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:14:24.095 [ INFO] Requesting file to download (this can take a while)...
2023-06-23 17:14:29.515 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:14:35.648 [ INFO] The product is ready for download
2023-06-23 17:14:35.648 [ INFO] Downloading file (this can take a while)...
2023-06-23 17:14:35.833 [ INFO] File type: application/x-netcdf
2023-06-23 17:14:35.833 [ INFO] File size: 127.7 kB (127732 B)
2023-06-23 17:14:35.833 [ INFO] Downloading file /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/mlotst/VARS_mlotst_DATES_1992-12-31_1993-12-31_LONS_144_149_LATS_-16_-23_DEPTHS_0-3_1.nc
2023-06-23 17:14:35.863 [ INFO] - 127.7 kB (51.3%)
2023-06-23 17:14:35.922 [ INFO] - 127.7 kB (100.0%)
2023-06-23 17:14:35.922 [ IN

2023-06-23 17:14:37.562 [ INFO] Asynchronous mode set
2023-06-23 17:14:37.562 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:14:38.279 [ INFO] Requesting file to download (this can take a while)...
2023-06-23 17:14:43.694 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:14:50.123 [ INFO] The product is ready for download
2023-06-23 17:14:50.124 [ INFO] Downloading file (this can take a while)...
2023-06-23 17:14:50.303 [ INFO] File type: application/x-netcdf
2023-06-23 17:14:50.303 [ INFO] File size: 127.7 kB (127732 B)
2023-06-23 17:14:50.303 [ INFO] Downloading file /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/mlotst/VARS_mlotst_DATES_1993-12-31_1994-12-31_LONS_144_149_LATS_-16_-23_DEPTHS_0-3_1.nc
2023-06-23 17:14:50.332 [ INFO] - 127.7 kB (51.3%)
2023-06-23 17:14:50.361 [ INFO] - 127.7 kB (100.0%)
2023-06-23 17:14:50.361 [ IN

2023-06-23 17:14:52.012 [ INFO] Asynchronous mode set
2023-06-23 17:14:52.012 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:14:52.712 [ INFO] Requesting file to download (this can take a while)...
2023-06-23 17:14:58.131 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:15:04.569 [ INFO] The product is ready for download
2023-06-23 17:15:04.569 [ INFO] Downloading file (this can take a while)...
2023-06-23 17:15:04.720 [ INFO] File type: application/x-netcdf
2023-06-23 17:15:04.720 [ INFO] File size: 127.7 kB (127732 B)
2023-06-23 17:15:04.720 [ INFO] Downloading file /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/mlotst/VARS_mlotst_DATES_1994-12-31_1995-12-31_LONS_144_149_LATS_-16_-23_DEPTHS_0-3_1.nc
2023-06-23 17:15:04.778 [ INFO] - 127.7 kB (51.3%)
2023-06-23 17:15:04.809 [ INFO] - 127.7 kB (100.0%)
2023-06-23 17:15:04.809 [ IN

2023-06-23 17:15:06.456 [ INFO] Asynchronous mode set
2023-06-23 17:15:06.457 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:15:07.182 [ INFO] Requesting file to download (this can take a while)...
2023-06-23 17:15:12.615 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 17:15:18.738 [ INFO] The product is ready for download
2023-06-23 17:15:18.738 [ INFO] Downloading file (this can take a while)...
2023-06-23 17:15:18.895 [ INFO] File type: application/x-netcdf
2023-06-23 17:15:18.895 [ INFO] File size: 127.7 kB (127732 B)
2023-06-23 17:15:18.895 [ INFO] Downloading file /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/mlotst/VARS_mlotst_DATES_1995-12-31_1996-12-31_LONS_144_149_LATS_-16_-23_DEPTHS_0-3_1.nc
2023-06-23 17:15:18.954 [ INFO] - 127.7 kB (51.3%)
2023-06-23 17:15:18.984 [ INFO] - 127.7 kB (100.0%)
2023-06-23 17:15:18.984 [ IN

...

2023-06-23 18:09:12.602 [ INFO] Asynchronous mode set
2023-06-23 18:09:12.602 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 18:09:13.308 [ INFO] Requesting file to download (this can take a while)...
2023-06-23 18:09:19.332 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 18:09:25.715 [ INFO] The product is ready for download
2023-06-23 18:09:25.715 [ INFO] Downloading file (this can take a while)...
2023-06-23 18:09:25.903 [ INFO] File type: application/x-netcdf
2023-06-23 18:09:25.903 [ INFO] File size: 128.1 kB (128104 B)
2023-06-23 18:09:25.903 [ INFO] Downloading file /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/vo/VARS_vo_DATES_2017-12-31_2018-12-31_LONS_144_149_LATS_-16_-23_DEPTHS_0-3_1.nc
2023-06-23 18:09:25.935 [ INFO] - 128.1 kB (51.2%)
2023-06-23 18:09:25.966 [ INFO] - 128.1 kB (100.0%)
2023-06-23 18:09:25.966 [ INFO] Proc

2023-06-23 18:09:27.589 [ INFO] Asynchronous mode set
2023-06-23 18:09:27.590 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 18:09:28.297 [ INFO] Requesting file to download (this can take a while)...
2023-06-23 18:09:33.715 [ INFO] Authenticating user otimmerman for service https://my.cmems-du.eu/motu-web/Motu
2023-06-23 18:09:40.431 [ INFO] The product is ready for download
2023-06-23 18:09:40.431 [ INFO] Downloading file (this can take a while)...
2023-06-23 18:09:40.612 [ INFO] File type: application/x-netcdf
2023-06-23 18:09:40.612 [ INFO] File size: 128.1 kB (128104 B)
2023-06-23 18:09:40.612 [ INFO] Downloading file /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/vo/VARS_vo_DATES_2018-12-31_2019-12-31_LONS_144_149_LATS_-16_-23_DEPTHS_0-3_1.nc
2023-06-23 18:09:40.643 [ INFO] - 128.1 kB (51.2%)
2023-06-23 18:09:40.672 [ INFO] - 128.1 kB (100.0%)
2023-06-23 18:09:40.672 [ INFO] Proc

 variable loop: 100%|██████████| 7/7 [55:19<00:00, 474.18s/it]

Dictionary saved as json file at /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/merged_vars/VARS_vo_DATES_1992-12-31_2020-12-16_LONS_144_149_LATS_-16_-23_DEPTHS_0-3_1.json
Writing cmems_gopr_monthly_B to file at /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/cmems_gopr_monthly_B.nc


Writing complete.
Dictionary saved as json file at /lustre_scratch/orlando-code/datasets/global_ocean_reanalysis/monthly_means/Great_Barrier_Reef_B/cmems_gopr_monthly_B.json


In [ ]:
spatial_plots.plot_spatial(xa.open_dataset(directories.get_monthly_cmems_dir() / "Great_Barrier_Reef_C/cmems_gopr_monthly_B.nc")["bottomT"].isel(time=0))

In [ ]:
# download daily data
xa_cmems_daily, cmems_daily_path = climate_data.download_reanalysis(download_dir=directories.get_daily_cmems_dir(), 
    final_filename = "cmems_gopr_daily.nc",
    lat_lims = reef_areas.get_lat_lon_limits(area_name)[0], lon_lims = reef_areas.get_lat_lon_limits(area_name)[1], 
    product_id = "cmems_mod_glo_phy_my_0.083_P1D-m")   

### Spatially pad the data

TODO: add visual explanation

In [ ]:
def spatially_buffer_timeseries(
    xa_ds: xa.Dataset,
    buffer_size: int = 3,
    exclude_vars: list[str] = ["spatial_ref", "coral_algae_1-12_degree"],
) -> xa.Dataset:
    """Applies a spatial buffer to each data variable in the xarray dataset.

    Parameters
        xa_ds (xarray.Dataset): Input xarray dataset.
        buffer_size (int): Buffer size in grid cells.
        exclude_vars (list[str]): List of variable names to exclude from buffering.

    Returns:
        xarray.Dataset: Xarray dataset with buffered data variables.
    """
    filtered_vars = [var for var in xa_ds.data_vars if var not in exclude_vars]

    buffered_ds = xa.Dataset()
    for data_var in tqdm(
        filtered_vars, desc=f"Buffering variables by {buffer_size} pixel(s)"
    ):
        buffered = xa.apply_ufunc(
            spatial_data.buffer_nans,
            xa_ds[data_var],
            input_core_dims=[[]],
            output_core_dims=[[]],
            kwargs={"size": buffer_size},
            dask="parallelized",
        )
        buffered_ds[data_var] = buffered

    return buffered_ds


def spatially_buffer_nc_file(nc_path: Path | str, buffer_size: int = 3):
    # TODO: specify distance buffer
    nc_path = Path(nc_path)
    buffered_name = nc_path.stem + f"_buffered_{buffer_size}_pixel"
    buffered_path = (nc_path.parent / buffered_name).with_suffix(".nc")

    # if buffered file doesn't already exist
    if not buffered_path.is_file():
        nc_file = xa.open_dataset(nc_path)
        buffered_ds = spatially_buffer_timeseries(
            nc_file, buffer_size=buffer_size
        )
        buffered_ds.to_netcdf(buffered_path)
    else:
        buffered_ds = xa.open_dataset(buffered_path)
        print(
            f"Area buffered by {buffer_size} pixel(s) already exists at {buffered_path}."
        )

    return buffered_ds, buffered_path

In [ ]:
# xa_cmems_monthly_buffered, _ = spatial_data.spatially_buffer_nc_file(cmems_monthly_path, buffer_size=5)
xa_cmems_daily_buffered, _ = spatially_buffer_nc_file(cmems_daily_path, buffer_size=5)
# TODO: this taking forever (seemingly getting stuck after second variable)

In [ ]:
spatial_plots.plot_DEM(xa.open_dataset(directories.get_monthly_cmems_dir() / "cmems_gopr_monthly_buffered_5_pixel")["mlotst"].isel(time=0), "")
# spatial_plots.plot_DEM(buffered["mlotst"].isel(time=0), "")

## Load ERA5 data

In [ ]:
climate_data.generate_era5_data(
    lat_lims=reef_areas.get_lat_lon_limits(area_name)[0], lon_lims=reef_areas.get_lat_lon_limits(area_name)[1])

### Congratulations!

